This notebook extracts all spelling and puncutation features from the set of queries found in SQS.

# Load Libraries

The following block of code loads all libraries needed for this notebook.

In [1]:
import csv
import pickle
import string
import textstat

import pandas as pd
import numpy as np

from langdetect import detect
from spellchecker import SpellChecker
from tqdm import tqdm

# Load Data Sets

This block of code loads the data sets and extracts all unique queries from both.

In [2]:
# allSessions = pickle.load( open( "../Data/DataSets/SWC/SWC.p", "rb" ) )
# allSessionsSQS = pickle.load( open( "../Data/DataSets/SQS/SQS.p", "rb" ) )
# allQueries = allSessions['query'].tolist() + allSessionsSQS['query'].tolist()
# allQueries = set(allQueries)

# allSessionsSQS = pickle.load( open( "../Data/DataSets/SQS/SQS.p", "rb" ) )
# allQueries = allSessionsSQS['query'].tolist()
# allQueries = np.array(allQueries)
# allQueries

allSessionsSQS = pickle.load( open( "../Data/DataSets/SQS/SQS.p", "rb" ) )
allQueries = allSessionsSQS['query'].tolist()
setQueries = allQueries


In [3]:
allSessionsSQS

,query,class,sID
0,becoming a fireman,0,3199
1,hotel in Pocono Mountains,0,2515
2,wedding traditions buddhism,0,2823
3,diversification in hiring,0,3033
4,traiditional swahili recipes,0,3145
...,...,...,...
1500,Who plays the bad guy in Star Wars the Horde a...,1,3256
1501,What is a fox's favorite kind of food?,1,2859
1502,"Show me the movie called ""The Martian""",1,3208
1503,What is the biggest rock found on Mars?,1,2676


In [4]:
len(setQueries)

1505

# Generate Misspelled List

Generates a list of commonly misspelled words by children from the KidSpell data set which is later used.

In [5]:
kidsMispelled = []

count = 0

with open('KidSpell/Web_Search_Lab_Errors.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        if count == 0:
            count += 1
        else:
            kidsMispelled.append(row[0])

count = 0
with open('KidSpell/Web_Search_Informal_Errors.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        if count == 0:
            count += 1
        else:
            kidsMispelled.append(row[0])
    
count = 0
with open('KidSpell/Essay_Writing_Errors.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        if count == 0:
            count += 1
        else:
            kidsMispelled.append(row[1])

kidsMispelled = set(kidsMispelled)

In [6]:
len(kidsMispelled)

1134

# Extract Spelling Features

The following block of code extracts features related to spelling errors and stores them in a dataframe. 

In [7]:
spell = SpellChecker()

spellingError = []
oneOffError = []
kidsError = []
misspelledCol = []

netModifiers = ['www', 'http', '.com', '.net', '.edu', '.org', '.gov', '.co', '.mil', '.com']

with tqdm(total = len(allQueries) ) as pbar:
    for query in allQueries:
        query =  query.translate(str.maketrans('', '', string.punctuation)) # -- remove all panctuations
        website = [mod for mod in netModifiers if(mod in query)] # --- save the netModifiers in the queries
        if not website:
            misspelledWords = "";
            try:
                lang = detect(query) # -- detects a language 
                misspelled = spell.unknown(query.split(" ")) # -- unknown gives the misspelled words
                found = 0
                oneOff = 0
                kidsMis = 0
                for word in misspelled:
                    misspelledWords += ", " + word # -- concatinating the misspelled words
                    if word in kidsMispelled:
                        kidsMis +=1
                    candid = spell.candidates(word) # candidates() displays the set of words that are close to the word entered
#                     print(candid)
#                     print("-------start edits-----------------")
                    edits = spell.edit_distance_1(word)
#                     print(edits)
#                     print("-----------end edits-------------")
                    
                    for can in candid:
                        if can in edits:
                            oneOff += 1
                        break
#                     print(word +' Out')
                oneOffError.append(oneOff)
                spellingError.append(len(misspelled))
                kidsError.append(kidsMis)
#                 misspelledCol.append(misspelledWords)
                
            except:
#                 print(misspelled)
                oneOffError.append(-1)
                spellingError.append(-1)
                kidsError.append(-1)
#                 misspelledCol.append(misspelledWords)

        else:
            spellingError.append(0)
            oneOffError.append(0)
            kidsError.append(0)
#             misspelledCol.append("none")

        pbar.update()
    
spelling = pd.DataFrame(data=spellingError, columns = ['numSpellingErrors'])
spelling['query'] = allQueries
spelling['offByOne'] = oneOffError
spelling['kidsError'] = kidsError
# spelling['ms'] = misspelledCol


100%|██████████| 1505/1505 [00:46<00:00, 32.43it/s]


In [8]:
# # ---- CHECKING ----

# spell = SpellChecker()

# spellingError = []
# oneOffError = []
# kidsError = []
# misspelledCol = []

# netModifiers = ['www', 'http', '.com', '.net', '.edu', '.org', '.gov', '.co', '.mil', '.com']

# with tqdm(total = len(allQueries) ) as pbar:
#     for query in allQueries:
#         query =  query.translate(str.maketrans('', '', string.punctuation)) # -- remove all panctuations
#         website = [mod for mod in netModifiers if(mod in query)] # --- save the netModifiers in the queries
#         if not website:
#             misspelledWords = "";
#             try:
#                 lang = detect(query) # -- detects a language 
#                 misspelled = spell.unknown(query.split(" ")) # -- unknown gives the misspelled words
#                 found = 0
#                 oneOff = 0
#                 kidsMis = 0
#                 for word in misspelled:
#                     misspelledWords += " " + word # -- concatinating the misspelled words
#                     if word in kidsMispelled:
#                         kidsMis +=1
#                     candid = spell.candidates(word) # --- candidates() displays the set of words that are close to the word entered e.g: 'country', 'count', 'counter' if country is entered
#                     edits = spell.edit_distance_1(word) # -- Compute all strings that are one edit away from `word` 
                    
#                     for can in candid:
#                         if can in edits:
#                             oneOff += 1 # -- one letter off 
#                         break

#                 oneOffError.append(oneOff)
#                 spellingError.append(len(misspelled))
#                 kidsError.append(kidsMis)
#                 misspelledCol.append(misspelledWords)
                
#             except:
# #                 print(misspelled)
#                 oneOffError.append(-1)
#                 spellingError.append(-1)
#                 kidsError.append(-1)
#                 misspelledCol.append(misspelledWords)

#         else:
#             spellingError.append(0)
#             oneOffError.append(0)
#             kidsError.append(0)
#             misspelledCol.append("none")

#         pbar.update()
    
# spelling = pd.DataFrame(data=spellingError, columns = ['numSpellingErrors'])
# spelling['query'] = allQueries
# spelling['offByOne'] = oneOffError
# spelling['kidsError'] = kidsError
# spelling['ms'] = misspelledCol


In [9]:
jj= 'ths is intentioal'

print(detect(jj))

kk=spell.unknown(jj.split(" "))
kk

en


{'intentioal', 'ths'}

In [10]:
candid = spell.candidates('ths') # candidates() displays the set of words that are close to the word entered
#                     print(candid)
#                     print("-------start edits-----------------")
edits = spell.edit_distance_1('ths')
#                     print(edits)
#                     print("-----------end edits-------------")
oneOff=0              
for can in candid:
    if can in edits:
        oneOff += 1
    break
oneOff

1

In [11]:
spell.candidates('ths')

{'tas',
 'tes',
 'tha',
 'thas',
 'the',
 'thes',
 'thi',
 'this',
 'tho',
 'thos',
 'thu',
 'thus',
 'thy',
 'tis',
 'tus'}

In [12]:
for j in range(5):
    print(j)
    if j ==0:
        print(j)
    break

0
0


In [13]:
spell.edit_distance_1('ths')

{"'hs",
 "'ths",
 'ahs',
 'aths',
 'bhs',
 'bths',
 'chs',
 'cths',
 'dhs',
 'dths',
 'ehs',
 'eths',
 'fhs',
 'fths',
 'ghs',
 'gths',
 'hhs',
 'hs',
 'hths',
 'hts',
 'ihs',
 'iths',
 'jhs',
 'jths',
 'khs',
 'kths',
 'lhs',
 'lths',
 'mhs',
 'mths',
 'nhs',
 'nths',
 'ohs',
 'oths',
 'phs',
 'pths',
 'qhs',
 'qths',
 'rhs',
 'rths',
 'shs',
 'sths',
 "t'hs",
 "t's",
 'tahs',
 'tas',
 'tbhs',
 'tbs',
 'tchs',
 'tcs',
 'tdhs',
 'tds',
 'tehs',
 'tes',
 'tfhs',
 'tfs',
 'tghs',
 'tgs',
 'th',
 "th'",
 "th's",
 'tha',
 'thas',
 'thb',
 'thbs',
 'thc',
 'thcs',
 'thd',
 'thds',
 'the',
 'thes',
 'thf',
 'thfs',
 'thg',
 'thgs',
 'thh',
 'thhs',
 'thi',
 'this',
 'thj',
 'thjs',
 'thk',
 'thks',
 'thl',
 'thls',
 'thm',
 'thms',
 'thn',
 'thns',
 'tho',
 'thos',
 'thp',
 'thps',
 'thq',
 'thqs',
 'thr',
 'thrs',
 'ths',
 "ths'",
 'thsa',
 'thsb',
 'thsc',
 'thsd',
 'thse',
 'thsf',
 'thsg',
 'thsh',
 'thsi',
 'thsj',
 'thsk',
 'thsl',
 'thsm',
 'thsn',
 'thso',
 'thsp',
 'thsq',
 'thsr',


In [14]:
'ths' in spell.edit_distance_1('ths')

True

In [15]:
for j in kk:
    print(spell.candidates(j))
#     print(spell.edit_distance_1(j))
    pass

{'intentional'}
{'tha', 'tis', 'thy', 'thi', 'thas', 'thos', 'tus', 'thus', 'thu', 'tes', 'the', 'this', 'tas', 'tho', 'thes'}


In [16]:
'' in kidsMispelled

True

In [17]:
spell = SpellChecker()

spellingError = []
oneOffError = []
kidsError = []
misspelledCol = []

netModifiers = ['www', 'http', '.com', '.net', '.edu', '.org', '.gov', '.co', '.mil', '.com']

with tqdm(total = len(allQueries) ) as pbar:
    for query in allQueries:
        query =  query.translate(str.maketrans('', '', string.punctuation)) # -- remove all panctuations
        website = [mod for mod in netModifiers if(mod in query)] # --- save the netModifiers in the queries
        if not website:
            misspelledWords = "";
            try:
                lang = detect(query) # -- detects a language 
                misspelled = spell.unknown(query.split(" ")) # -- unknown gives the misspelled words
                found = 0
                oneOff = 0
                kidsMis = 0
                for word in misspelled:
                    misspelledWords += ", " + word # -- concatinating the misspelled words
                    if word in kidsMispelled:
                        kidsMis +=1
                    candid = spell.candidates(word) # candidates() displays the set of words that are close to the word entered
#                     print(candid)
#                     print("-------start edits-----------------")
                    edits = spell.edit_distance_1(word)
#                     print(edits)
#                     print("-----------end edits-------------")
                    
                    for can in candid:
                        if can in edits:
                            oneOff += 1
                        break
#                     print(word +' Out')
                oneOffError.append(oneOff)
                spellingError.append(len(misspelled))
                kidsError.append(kidsMis)
#                 misspelledCol.append(misspelledWords)
                
            except:
#                 print(misspelled)
                oneOffError.append(-1)
                spellingError.append(-1)
                kidsError.append(-1)
#                 misspelledCol.append(misspelledWords)

        else:
            spellingError.append(0)
            oneOffError.append(0)
            kidsError.append(0)
#             misspelledCol.append("none")

        pbar.update()
    
spelling = pd.DataFrame(data=spellingError, columns = ['numSpellingErrors'])
spelling['query'] = allQueries
spelling['offByOne'] = oneOffError
spelling['kidsError'] = kidsError
# spelling['ms'] = misspelledCol


100%|██████████| 1505/1505 [00:44<00:00, 34.12it/s]


In [18]:
allQueries[7]

'efficiency of solar panels'

In [19]:
' ' in allQueries

False

In [20]:
for can in candid:
                        if can in edits:
                            oneOff += 1
                        break

In [21]:
spell = SpellChecker()

spellingError = [] # number of misspelled words in the query
oneOffError = []
kidsError = []  # number of erros found in kidsMispelled
misspelledCol = []
allErrors = [] # list of concatenated errors per query


netModifiers = ['www', 'http', '.com', '.net', '.edu', '.org', '.gov', '.co', '.mil', '.com']
with tqdm(total = len(allQueries) ) as pbar:
    for query in allQueries:
        query =  query.translate(str.maketrans('', '', string.punctuation)) # -- remove all panctuations
        website = [mod for mod in netModifiers if(mod in query)] # --- save the netModifiers in the queries
        if not website:
            misspelledWords = "";
            try:
                lang = detect(query) # -- detects a language 
                misspelled = spell.unknown(query.split(" ")) # -- unknown gives the misspelled words
                spellingError.append(len(misspelled))
                
                all_error = ', '.join(misspelled)
                allErrors.append(all_error)
                
                kidsMis = 0
                #oneoff =0
                for word in misspelled:
                    if word in kidsMispelled:
                        kidsMis +=1
                        
                    candid = spell.candidates(word)
                    edits = spell.edit_distance_1(word)
                    
#                     if list(candid)[0] in edits:
#                         oneoff+=1
# #                     for can in candid:
#                         if can in edits:
#                             oneoff+=1
#                         break
                        
                
                kidsError.append(kidsMis)
                #oneOffError.append(oneoff)
            
            except:
                spellingError.append(-1)
                kidsError.append(-1)
                allErrors.append(-1)
                #oneOffError.append(-1)
            
        else:
            spellingError.append(0)
            kidsError.append(0)
            allErrors.append(0)
            #oneOffError.append(0)

        pbar.update()

100%|██████████| 1505/1505 [00:44<00:00, 34.18it/s]


In [22]:
len(allErrors)

1505

In [23]:
df =pd.DataFrame({
    'Query': allQueries,
    'kidsError': kidsError,
    'numSpellingErrors': spellingError,
    'ms': allErrors,
    #'offByOne': oneOffError
})
df

,Query,kidsError,numSpellingErrors,ms
0,becoming a fireman,0,0,
1,hotel in Pocono Mountains,0,1,pocono
2,wedding traditions buddhism,0,0,
3,diversification in hiring,0,0,
4,traiditional swahili recipes,0,2,"swahili, traiditional"
...,...,...,...,...
1500,Who plays the bad guy in Star Wars the Horde a...,0,1,awakends
1501,What is a fox's favorite kind of food?,0,1,foxs
1502,"Show me the movie called ""The Martian""",0,0,
1503,What is the biggest rock found on Mars?,0,0,


In [24]:
'jp' in spell.edit_distance_1('jp')

True

In [25]:
df.query('numSpellingErrors > 1')

,Query,kidsError,numSpellingErrors,ms
4,traiditional swahili recipes,0,2,"swahili, traiditional"
11,regular phones vs internet phones voip experie...,0,2,"vs, voip"
21,Pocono Mountains things to do,1,2,", pocono"
83,bioinformatics degree SUNY,0,2,"suny, bioinformatics"
88,directions from OKC to the pocono mountains,0,2,"pocono, okc"
...,...,...,...,...
1388,Who plays Kylo Ren from StarWars?,0,2,"starwars, kylo"
1400,How old is Johny enlgish?,0,2,"johny, enlgish"
1431,How long can a polr bear swin under water?,1,2,"polr, swin"
1456,Batman vs superman,1,2,", vs"


In [26]:
len(allQueries)

1505

In [27]:
# spell = SpellChecker()

# spellingError = []
# oneOffError = []
# kidsError = []
# misspelledCol = []


# netModifiers = ['www', 'http', '.com', '.net', '.edu', '.org', '.gov', '.co', '.mil', '.com']
# with tqdm(total = len(allQueries) ) as pbar:
#     for query in allQueries:
#         query =  query.translate(str.maketrans('', '', string.punctuation)) # -- remove all panctuations
#         website = [mod for mod in netModifiers if(mod in query)] # --- save the netModifiers in the queries
#         if not website:
#             misspelledWords = "";
            
#             try:
#                 lang = detect(query) # -- detects a language 
#                 misspelled = spell.unknown(query.split(" ")) # -- unknown gives the misspelled words
                

In [28]:
edits = spell.edit_distance_1(word)

In [29]:
spell = SpellChecker()

spellingError = []
oneOffError = []
kidsError = []
misspelledCol = []

test = ['it consulting',
 'JP Morgan Chase data',
 'fun violence control jp constitutional']
test.append('we add www and http jp intentionally')
netModifiers = ['www', 'http', '.com', '.net', '.edu', '.org', '.gov', '.co', '.mil', '.com']

for query in test:
    query =  query.translate(str.maketrans('', '', string.punctuation)) # -- remove all panctuations
    website = [mod for mod in netModifiers if(mod in query)]
    
    if not website:
        misspelledWords = "";
        
        lang = detect(query) # -- detects a language 
        print(query)
        misspelled = spell.unknown(query.split(" ")) # -- unknown gives the misspelled words
        print('length of',len(misspelled))
        found = 0
        oneOff = 0
        kidsMis = 0
        for word in misspelled:
            misspelledWords += " " + word  # -- concatinating the misspelled words
            print('word = ', list(word))
            print(list(misspelledWords))
            if word in kidsMispelled:
                kidsMis +=1
            candid = spell.candidates(word) # --- candidates() displays the set of words that are close to the word entered e.g: 'country', 'count', 'counter' if country is entered
            edits = spell.edit_distance_1(word) # -- Compute all strings that are one edit away from `word` 
                    
            for can in candid:
                if can in edits:
                    oneOff += 1 # -- one letter off 
                break

        oneOffError.append(oneOff)
        #print(oneOffError)
        print('final', misspelled)
        spellingError.append(len(misspelled))
        #print(spellingError)
        kidsError.append(kidsMis)
        #print(kidsError)
        misspelledCol.append(misspelledWords)
        print(misspelledCol)

it consulting
length of 0
final set()
['']
JP Morgan Chase data
length of 1
word =  ['j', 'p']
[' ', 'j', 'p']
final {'jp'}
['', ' jp']
fun violence control jp constitutional
length of 1
word =  ['j', 'p']
[' ', 'j', 'p']
final {'jp'}
['', ' jp', ' jp']


In [30]:
'' + ' ' + 'A'

' A'

In [31]:
website

['www', 'http']

In [32]:
# ---- CHECKING ----

spell = SpellChecker()

spellingError = []
oneOffError = []
kidsError = []
misspelledCol = []

netModifiers = ['www', 'http', '.com', '.net', '.edu', '.org', '.gov', '.co', '.mil', '.com']

with tqdm(total = len(allQueries) ) as pbar:
    for query in allQueries:
        query =  query.translate(str.maketrans('', '', string.punctuation)) # -- remove all panctuations
        website = [mod for mod in netModifiers if(mod in query)] # --- save the netModifiers in the queries
        if not website:
            misspelledWords = "";
            try:
                lang = detect(query) # -- detects a language 
                misspelled = spell.unknown(query.split(" ")) # -- unknown gives the misspelled words
                found = 0
                oneOff = 0
                kidsMis = 0
                for word in misspelled:
                    misspelledWords += " " + word # -- concatinating the misspelled words
                    
                    misspelledWords = ' '.join(misspelledWords.split())
                    
                    if word in kidsMispelled:
                        kidsMis +=1
                    candid = spell.candidates(word) # --- candidates() displays the set of words that are close to the word entered e.g: 'country', 'count', 'counter' if country is entered
                    edits = spell.edit_distance_1(word) # -- Compute all strings that are one edit away from `word` 
                    
                    for can in candid:
                        if can in edits:
                            oneOff += 1 # -- one letter off 
                        break

                oneOffError.append(oneOff)
                print('mispelled', misspelled)
                spellingError.append(len(misspelled))
                kidsError.append(kidsMis)
                misspelledCol.append(misspelledWords)
                
            except:
#                 print(misspelled)
                oneOffError.append(-1)
                spellingError.append(-1)
                kidsError.append(-1)
                misspelledCol.append(misspelledWords)

        else:
            spellingError.append(0)
            oneOffError.append(0)
            kidsError.append(0)
            misspelledCol.append("none")

        pbar.update()
    
spelling = pd.DataFrame(data=spellingError, columns = ['numSpellingErrors'])
spelling['query'] = allQueries
spelling['offByOne'] = oneOffError
spelling['kidsError'] = kidsError
# spelling['ms'] = misspelledCol


  0%|          | 0/1505 [00:00<?, ?it/s]

mispelled set()
mispelled {'pocono'}
mispelled set()
mispelled set()


  0%|          | 5/1505 [00:00<00:37, 40.16it/s]

mispelled {'swahili', 'traiditional'}
mispelled {'boolywood'}
mispelled set()
mispelled set()
mispelled {'bollywoods'}
mispelled set()
mispelled set()
mispelled {'vs', 'voip'}
mispelled set()
mispelled set()
mispelled set()


  1%|          | 16/1505 [00:00<00:27, 54.83it/s]

...

mispelled {'starwars'}
mispelled set()
mispelled set()
mispelled set()
mispelled {'girrafe'}
mispelled set()


 98%|█████████▊| 1475/1505 [00:43<00:00, 30.67it/s]

mispelled set()
mispelled set()
mispelled set()
mispelled {'starwars'}
mispelled set()
mispelled set()
mispelled set()


100%|██████████| 1505/1505 [00:43<00:00, 34.45it/s]

mispelled set()
mispelled set()
mispelled {'', 'bagan', 'billbow'}
mispelled set()
mispelled set()
mispelled set()
mispelled set()
mispelled {'mutan'}
mispelled set()
mispelled set()
mispelled set()
mispelled set()
mispelled set()
mispelled set()
mispelled {'kylo'}
mispelled set()
mispelled {''}
mispelled set()
mispelled set()
mispelled set()
mispelled set()
mispelled {'awakends'}
mispelled {'foxs'}
mispelled set()
mispelled set()
mispelled set()


In [33]:
spelling.head(15)

,numSpellingErrors,query,offByOne,kidsError
0,0,becoming a fireman,0,0
1,1,hotel in Pocono Mountains,1,0
2,0,wedding traditions buddhism,0,0
3,0,diversification in hiring,0,0
4,2,traiditional swahili recipes,1,0
5,1,Boolywood in hollywood,1,0
6,0,week long vacation idea,0,0
7,0,efficiency of solar panels,0,0
8,1,bollywood's increasing popularity,1,0
9,0,New York City,0,0


In [34]:
cc = 'You hate a ddog'
m = spell.unknown(cc.split(" "))
m

{'ddog'}

In [35]:
dd = 'how god exsists'
m = spell.unknown(dd.split(" "))
m

{'exsists'}

In [36]:
spelling

,numSpellingErrors,query,offByOne,kidsError
0,0,becoming a fireman,0,0
1,1,hotel in Pocono Mountains,1,0
2,0,wedding traditions buddhism,0,0
3,0,diversification in hiring,0,0
4,2,traiditional swahili recipes,1,0
...,...,...,...,...
1500,1,Who plays the bad guy in Star Wars the Horde a...,1,0
1501,1,What is a fox's favorite kind of food?,1,0
1502,0,"Show me the movie called ""The Martian""",0,0
1503,0,What is the biggest rock found on Mars?,0,0


In [37]:
spelling['kidsError'].unique()

array([ 0, -1,  1])

In [38]:
# spelling.ms[631]

In [39]:
spelling[spelling['kidsError']==1]

,numSpellingErrors,query,offByOne,kidsError
21,2,Pocono Mountains things to do,2,1
22,1,Connecticut Fire Academy,1,1
76,1,can a person continue sport after getting col...,1,1
84,1,what is collagen disease,1,1
110,1,"homicide "" martin bryant""",1,1
167,1,US government spending +space exploration,1,1
179,1,chateau resort getting to,1,1
184,2,Kenya Swahili dishes,1,1
217,1,employee evaluation tips,1,1
223,1,road trip + vacation,1,1


In [40]:
# len(s.edit_distance_2('world'))

In [41]:
q = "what is the nme of www"
netModifiers = ['www', 'http', '.com', '.net', '.edu', '.org', '.gov', '.co', '.mil', '.com']
w = [mod for mod in netModifiers if(mod in q)]
w

['www']

In [42]:
lan = 'oppure'
detect(lan)

'no'

In [43]:
s = SpellChecker()

mis = s.unknown(['countr', 'hapen'])


for w in mis:
    print(w, s.correction(w))
    
    print(s.candidates(w))
 

hapen happen
{'hagen', 'hacen', 'haven', 'happen', 'haben'}
countr country
{'count', 'counts', 'country', 'county', 'counter'}


In [44]:
[w for w in mis]

['hapen', 'countr']

In [45]:
print(word)

foxs


In [46]:
d = 'Ibra, and Micheal!' 
print(d.translate(str.maketrans('', '', string.punctuation)))

Ibra and Micheal


# Extract Punctuation And Casing Features

The following block of code extracts spelling and casing features before adding them to the dataframe. 

In [47]:
invalidcharacters= set(['!', ',', '.', '?'])
punct = []
casing = []
with tqdm(total = len(allQueries) ) as pbar:
    for query in allQueries:

        if any(char in invalidcharacters for char in query):
            if any(substring in query for substring in netModifiers):
                punct.append(0)
            else:
                punct.append(1)
        else:
            punct.append(0)

        if query.islower():
            casing.append(0)
        else:
            casing.append(1)
        pbar.update()
        
spelling['punct'] = punct
spelling['casing'] = casing



100%|██████████| 1505/1505 [00:00<00:00, 475512.43it/s]


In [48]:
spelling

,numSpellingErrors,query,offByOne,kidsError,punct,casing
0,0,becoming a fireman,0,0,0,0
1,1,hotel in Pocono Mountains,1,0,0,1
2,0,wedding traditions buddhism,0,0,0,0
3,0,diversification in hiring,0,0,0,0
4,2,traiditional swahili recipes,1,0,0,0
...,...,...,...,...,...,...
1500,1,Who plays the bad guy in Star Wars the Horde a...,1,0,1,1
1501,1,What is a fox's favorite kind of food?,1,0,1,1
1502,0,"Show me the movie called ""The Martian""",0,0,0,1
1503,0,What is the biggest rock found on Mars?,0,0,1,1


# Return Feature Set

Returns dataframe with spelling and punctuation features.

In [49]:
pickle.dump(spelling, open( "Pickles/SPFeat.p", "wb" ) )

In [50]:
spelling.head(30)

,numSpellingErrors,query,offByOne,kidsError,punct,casing
0,0,becoming a fireman,0,0,0,0
1,1,hotel in Pocono Mountains,1,0,0,1
2,0,wedding traditions buddhism,0,0,0,0
3,0,diversification in hiring,0,0,0,0
4,2,traiditional swahili recipes,1,0,0,0
5,1,Boolywood in hollywood,1,0,0,1
6,0,week long vacation idea,0,0,0,0
7,0,efficiency of solar panels,0,0,0,0
8,1,bollywood's increasing popularity,1,0,0,0
9,0,New York City,0,0,0,1


In [51]:
spelling.shape

(1505, 6)

In [52]:
spelling.columns

Index(['numSpellingErrors', 'query', 'offByOne', 'kidsError', 'punct',
       'casing'],
      dtype='object')

In [53]:
print('done')

done


In [54]:
# expected shape: (1505, 6)